In [1]:
import glob
import regex as re
import numpy as np
import pandas as pd
import xgboost as xgb
import tensorflow as tf
import matplotlib.pyplot as plt
pd.options.display.float_format = '{:,.2f}'.format
%matplotlib inline
%load_ext rpy2.ipython
%run ../airquality/data/gen_daily_targets.py

/usr/local/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: During startup - 
  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Warning messages:

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: 1: Setting LC_COLLATE failed, using "C" 

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: 2: Setting LC_TIME failed, using "C" 

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: 3: Setting LC_MESSAGES failed, using "C" 

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: 4: Setting LC_MONETARY failed, using "C" 

  warnings.warn(x, RRuntimeWarning)


In [2]:
with open('../data/headers_mod.csv') as f:
    mod_headers = f.readlines()[0].split()
    
with open('../data/headers_obs.csv') as f:
    obs_headers = f.readlines()[0].split()
    
mod_headers, obs_headers

(['lon', 'lat', 'day', 'hour', 'AirPollutant', 'Concentration'],
 ['Countrycode',
  'Namespace',
  'AirQualityNetwork',
  'AirQualityStation',
  'AirQualityStationEoICode',
  'SamplingPoint',
  'SamplingProcess',
  'Sample',
  'AirPollutant',
  'AirPollutantCode',
  'AveragingTime',
  'Concentration',
  'UnitOfMeasurement',
  'DatetimeBegin',
  'DatetimeEnd',
  'Validity',
  'Verification'])

In [3]:
test = pd.read_csv('../data/targets.csv')
stations = pd.read_csv('../data/stations.csv')

In [4]:
observations = []
for folder in glob.glob('../data/obs/*'):
    for file in glob.glob('{}/*'.format(folder)):
        year = re.search('\d{4}', file)[0]
        station = 'ES{}A'.format(re.findall('\d{4}', file)[1])
        data = pd.read_csv(file, sep='\t', names=obs_headers)
        data['year'] = year
        data['station'] = station
        observations.append(data)
        
observations = pd.concat(observations)
observations = observations[[c for c in observations.columns if c not in [
    'Countrycode',
    'Namespace',
    'AirQualityNetwork',
    'AirQualityStation',
    'AirQualityStationEoICode',
    'SamplingPoint',
    'SamplingProcess',
    'AveragingTime',
    'UnitOfMeasurement',
    'AirPollutant',
    'AirPollutantCode',
    'Validity',
    'Verification',
    'DatetimeEnd',
    'Sample'
]]]
observations['datetime'] = pd.to_datetime(observations['DatetimeBegin'])
observations = observations.drop('DatetimeBegin', axis=1)
observations = observations.sort_values('datetime')
observations['day'] = observations['datetime'].apply(lambda dt: dt.date())
observations['time'] = observations['datetime'].apply(lambda dt: dt.time())
observations = pd.merge(observations, gen_daily_targets(observations)[['station', 'day', 'target']],
                        how='left', on=['station', 'day'])
observations.to_csv('../data/observations.csv')
observations.head(5)

,Concentration,year,station,datetime,day,time,target
0,19,2013,ES1992A,2013-01-01 01:00:00,2013-01-01,01:00:00,0
1,54,2013,ES1480A,2013-01-01 01:00:00,2013-01-01,01:00:00,0
2,11,2013,ES1856A,2013-01-01 01:00:00,2013-01-01,01:00:00,0
3,39,2013,ES1396A,2013-01-01 01:00:00,2013-01-01,01:00:00,0
4,55,2013,ES0691A,2013-01-01 01:00:00,2013-01-01,01:00:00,0


In [5]:
observations.groupby(['year', 'station']).agg({
    'Concentration': ['mean', 'std', 'var', 'count', 'min', 'max'],
    'datetime': ['min', 'max'],
    'target': ['sum']
})

Concentration                                        datetime  \
                      mean   std    var count min  max                 min   
year station                                                                 
2013 ES0691A         40.01 22.48 505.51  8643   1  135 2013-01-01 01:00:00   
     ES1396A         33.65 22.04 485.90  8627   1  163 2013-01-01 01:00:00   
     ES1438A         56.73 27.00 729.02  8420   1  253 2013-01-01 01:00:00   
     ES1480A         54.59 27.63 763.52  8656   1  229 2013-01-01 01:00:00   
     ES1679A         35.37 23.34 544.71  8625   1  183 2013-01-01 01:00:00   
     ES1856A         27.50 21.46 460.52  8496   1  147 2013-01-01 01:00:00   
     ES1992A         32.66 24.13 582.27  8617   1  183 2013-01-01 01:00:00   
2014 ES0691A         39.35 22.70 515.25  8585   3  141 2013-12-31 23:00:00   
     ES1396A         32.15 20.56 422.81  8592   3  158 2013-12-31 23:00:00   
     ES1438A         52.35 23.83 567.98  8562   7  170 2013-12-31 23:00:00   
     ES1480A         51.82 26.64 709.55  8592   3  202 2013-12-31 23:00:00   
     ES1679A         37.24 22.99 528.56  8660   1  132 2013-12-31 23:00:00   
     ES1856A         28.48 21.64 468.16  8613   1  157 2013-12-31 23:00:00   
     ES1992A         31.22 23.86 569.22  8597   1  189 2013-12-31 23:00:00   
2015 ES0691A         45.16 24.79 614.64  5691   4  166 2014-12-31 23:00:00   
     ES1396A         36.41 23.21 538.53  5469   1  172 2014-12-31 23:00:00   
     ES1438A         56.97 25.08 629.03  5617   8  178 2014-12-31 23:00:00   
     ES1480A         54.87 27.93 780.25  5554   1  217 2014-12-31 23:00:00   
     ES1679A         42.39 24.76 612.95  5745   1  172 2014-12-31 23:00:00   
     ES1856A         32.21 23.76 564.38  5750   1  169 2014-12-31 23:00:00   
     ES1992A         34.78 24.80 614.96  5622   1  195 2014-12-31 23:00:00   

                                 target  
                             max    sum  
year station                             
2013 ES0691A 2013-12-31 23:00:00     24  
     ES1396A 2013-12-31 23:00:00    168  
     ES1438A 2013-12-31 23:00:00    942  
     ES1480A 2013-12-31 23:00:00    662  
     ES1679A 2013-12-31 23:00:00     24  
     ES1856A 2013-12-31 23:00:00     24  
     ES1992A 2013-12-31 23:00:00    286  
2014 ES0691A 2014-12-31 23:00:00     70  
     ES1396A 2014-12-31 23:00:00    142  
     ES1438A 2014-12-31 23:00:00    404  
     ES1480A 2014-12-31 23:00:00    545  
     ES1679A 2014-12-31 23:00:00     24  
     ES1856A 2014-12-31 23:00:00    120  
     ES1992A 2014-12-31 23:00:00    223  
2015 ES0691A 2015-12-31 23:00:00     87  
     ES1396A 2015-12-31 23:00:00    119  
     ES1438A 2015-12-31 23:00:00    556  
     ES1480A 2015-12-31 23:00:00    668  
     ES1679A 2015-12-31 23:00:00     96  
     ES1856A 2015-12-31 23:00:00    144  
     ES1992A 2015-12-31 23:00:00    331

In [10]:
len(models)

7665

In [11]:
# models = []
# for folder in glob.glob('../data/mod/*'):
#     for file in glob.glob('{}/*'.format(folder)):
#         year = re.search('\d{4}', file)[0]
#         station = 'ES{}A'.format(re.findall('\d{4}', file)[-1])
#         data = pd.read_table(file, sep='\s+', names=mod_headers)
#         data['days_ahead'] = (data['day'] == data['day'].max()).astype(int)
#         days_ahead = data.pivot(index='hour', columns='days_ahead', values='Concentration').reset_index()
#         days_ahead['day'] = data.day.min()
#         data = pd.merge(days_ahead, data[['lon', 'lat', 'hour']],
#                         on='hour', how='left')\
#             .rename(columns={0: 'pred_0_days', 1: 'pred_1_days'})\
#             .drop_duplicates()
#         data['year'] = year
#         data['station'] = station
#         models.append(data)
        
models = pd.concat(models)
models['datetime'] = pd.to_datetime(models['day'].astype(str) + ' ' + models['hour'].astype(str))
models = models.sort_values('datetime', ascending=True)
models['day'] = models['datetime'].apply(lambda dt: dt.date())
models['time'] = models['datetime'].apply(lambda dt: dt.time())
models = models[[c for c in models.columns if c not in [
    'AirPollutant',
    'hour'
]]].sort_values('datetime')
models.to_csv('../data/models.csv')
models.head(5)

,pred_0_days,pred_1_days,day,lon,lat,year,station,datetime,time
0,38.79,34.11,2013-01-01,2.15,41.39,2013,ES1438A,2013-01-01,00:00:00
0,28.53,27.48,2013-01-01,2.13,41.38,2013,ES1396A,2013-01-01,00:00:00
0,35.85,42.57,2013-01-01,2.20,41.40,2013,ES0691A,2013-01-01,00:00:00
0,31.81,31.59,2013-01-01,2.15,41.40,2013,ES1480A,2013-01-01,00:00:00
0,31.81,31.59,2013-01-01,2.15,41.43,2013,ES1856A,2013-01-01,00:00:00


In [16]:
observations.shape

(159733, 7)

In [12]:
models.shape

(183960, 9)

In [ ]:
models.groupby(['year', 'station']).agg({
    'Concentration': ['mean', 'std', 'var', 'count', 'min', 'max'],
    'datetime': ['min', 'max']
})

In [ ]:
for s in observations['station'].unique():
    plt.figure(figsize=(20, 4))
    plt.hist(observations[observations['station'] == s]['Concentration'], bins=250)
    plt.figure(figsize=(20, 4))
    x_ax = list(range(0, observations[observations['station'] == s].shape[0]))
    plt.plot(x_ax, observations[observations['station'] == s]['Concentration'], 'r-', lw=.2)
    plt.plot(x_ax, observations[observations['station'] == s]['Concentration'], 'b--', lw=.1)

In [ ]:
observations.drop_duplicates(['station', 'datetime']).shape, models.drop_duplicates(['station', 'datetime']).shape